# Importing Libraries

In [16]:
#selenium libraries
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException, ElementClickInterceptedException

import time as time
import pandas as pd
import re

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

import requests
import urllib.parse
import os
import shutil

In [17]:
def get_drive_launch(url):
    """
    This function initiates the Chrome browser's driver with the url passed as a parameter and returns the driver instance.
    Parameters:
        url - url of the website
    return:
        driver - driver instance of the chrome browser    
    """
    #creating driver instance
    driver = webdriver.Chrome('./driver/chromedriver.exe')
    driver.maximize_window()
    
    #defining implicit wait
    driver.implicitly_wait(10)
    
    #launching the url
    driver.get(url)
    
    return driver

## 1.Scrape the details of most viewed videos on YouTube from Wikipedia:  
Url= https://en.wikipedia.org/wiki/List_of_most-viewed_YouTube_videos.  

You need to find following details:  

A) Rank 

B) Name 

C) Artist 

D) Upload date 

E) Views

In [18]:
def scrape_most_viewed_videos_youtube(url):
    table_MN = pd.read_html(url, match='Top 30 most-viewed YouTube videos')[0]
    table_MN.drop(['Note','Unnamed: 6'], axis = 1,inplace = True)
    table_MN.rename(columns={'No.':'Rank'},inplace=True)
    df = table_MN[:-1].copy()
    df['Video name'] = df['Video name'].apply(lambda x: x.split('[')[0])
    return df


In [19]:
url = "https://en.wikipedia.org/wiki/List_of_most-viewed_YouTube_videos"
data = scrape_most_viewed_videos_youtube(url)
data

,Rank,Video name,Uploader,Views (billions),Upload date
0,1.,"""Baby Shark Dance""",Pinkfong Kids' Songs & Stories,8.44,"June 17, 2016"
1,2.,"""Despacito""",Luis Fonsi,7.32,"January 12, 2017"
2,3.,"""Shape of You""",Ed Sheeran,5.29,"January 30, 2017"
3,4.,"""Johny Johny Yes Papa""",LooLoo Kids,5.24,"October 8, 2016"
4,5.,"""See You Again""",Wiz Khalifa,5.08,"April 6, 2015"
5,6.,"""Masha and the Bear – Recipe for Disaster""",Get Movies,4.43,"January 31, 2012"
6,7.,"""Uptown Funk""",Mark Ronson,4.16,"November 19, 2014"
7,8.,"""Gangnam Style""",Psy,4.05,"July 15, 2012"
8,9.,"""Learning Colors – Colorful Eggs on a Farm""",Miroshka TV,3.97,"February 27, 2018"
9,10.,"""Bath Song""",Cocomelon – Nursery Rhymes,3.95,"May 2, 2018"


## 2. Scrape the details team India’s international fixtures from bcci.tv.
Url = https://www.bcci.tv/.

You need to find following details:

A) Match title (I.e. 1st ODI)

B) Series

C) Place

D) Date

E) Time

Note: - From bcci.tv home page you have reach to the international fixture page through code

In [37]:
url = 'https://www.bcci.tv/'
d = get_drive_launch(url)
url = d.find_element_by_xpath("//div[contains(text(),'International')]/following-sibling::div//ul//a[contains(text(),'Fixtures')]").get_attribute('href')
d.quit()
d = get_drive_launch(url)

#time.sleep(3)

match_titles = []
series = []
places = []
dates = []
times = []

titles_elemns = d.find_elements_by_xpath("//strong[@class='fixture__name fixture__name--with-margin']")
series_elemns = d.find_elements_by_xpath("//div[@class='fixture__format-strip']")
places_elemns = d.find_elements_by_xpath("//strong[@class='fixture__name fixture__name--with-margin']/following-sibling::span")
dates_elemns = d.find_elements_by_xpath("//span[@class='fixture__date']")
months_elemns = d.find_elements_by_xpath("//span[@class='fixture__month']")
years_elemns = d.find_elements_by_xpath("//h3[@class='event-list__date js-date']")
times_elemns = d.find_elements_by_xpath("//span[@class='fixture__time']")
print(len(titles_elemns))
print(len(titles_elemns))
print(len(titles_elemns))
print(len(titles_elemns))

for title, series, place, date, month, time in zip(titles_elemns,series_elemns,places_elemns,dates_elemns,months_elemns, times_elemns):
    print(title.text)
    print(series.text.replace('\n','-'))
    print(place.text)
    yr = ''
    for i in years_elemns:
        if month.text.lower() in i.text.lower():
            yr = i.text
            break
    print(date.text+" "+yr)
    print(time.text)
    print("*"*20)

6
6
6
6
Final
TEST-ICC WORLD TEST CHAMPIONSHIP FINAL
The Ageas Bowl, Southampton
18 June 2021
15:30 IST
********************
1st Test
TEST-ENGLAND V INDIA 2021
Trent Bridge, Nottingham
04 August 2021
15:30 IST
********************
2nd Test
TEST-ENGLAND V INDIA 2021
Lord's, London
12 August 2021
15:30 IST
********************
3rd Test
TEST-ENGLAND V INDIA 2021
Headingley, Leeds
25 August 2021
15:30 IST
********************
4th Test
TEST-ENGLAND V INDIA 2021
The Oval, London
02 September 2021
15:30 IST
********************
5th Test
TEST-ENGLAND V INDIA 2021
Old Trafford, Manchester
10 September 2021
15:30 IST
********************
